<a href="https://colab.research.google.com/github/gintlomont/Projeto_imersao_alura/blob/main/projeto_Sistema_de_An%C3%A1lise_de_Dados_e_Conversa%C3%A7%C3%A3o_da_imersao_alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Análise de Dados e Conversação com Chat Utilizando Google Generative AI


*   Este código cria um sistema no console que oferece análise de dados e chatbot. Com a API do Google Generative AI, os usuários podem carregar arquivos CSV, JSON ou TXT para análise, fornecendo prompts para orientar a análise.
*   Além disso, podem conversar com um chatbot alimentado pela tecnologia de geração de linguagem natural do Google. Ideal para extrair insights de dados ou interagir com um chatbot avançado no console Python.

<br>
<br>
Exemplos: <br>
Voce pode usar o seu arquivo ou link da irmarsão (arquivo json)

1.   https://raw.githubusercontent.com/guilhermeonrails/api-imersao-ia/main/words.json
2.   Do que se trata este arquivo.
3.   Descreve este arquivo.
2.   Quais são os principais insights que podem ser extraídos deste dados.
4.   Pergunte o que quiser sobre os seus arquivo ou dados










#1.   Importar biblitecas




In [1]:
import pandas as pd
import google.generativeai as genai

# bibliteca que trabalha com formatação do texto
import textwrap
from IPython.display import display, Markdown

# bibliteca que limpa tela
from IPython.display import clear_output

# bibloteca time
from time import sleep

#2. Configurar a API do Google Generative AI

In [46]:
GOOGLE_API_KEY = 'Coloque o se API_kEY'
genai.configure(api_key=GOOGLE_API_KEY)

# configurar Genai
generation_config = {
  "candidate_count": 1,
  "temperature": 1,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }
model = genai.GenerativeModel('gemini-pro', safety_settings, generation_config, )

#3. Criando Funçoes importantes

## 3.1 Analisador de dados

In [45]:
def analisar_dados(dados, prompt=None):
    """Analisa os dados fornecidos usando o Gemini-pro e retorna os insights."""

    # Limitando a quantidade de linhas exibidas para evitar ultrapassar o numeros de tokens permitidos
    #dados = dados[150].to_string()

    # Preparar a entrada para o Gemini-pro
    texto_entrada = f"Dados: {dados}\nPrompt: {prompt}"

    # Gerar insights com o Gemini-pro
    resposta = chat.send_message(texto_entrada)
    insights = resposta.text

    # Retornar os insights
    return insights

## 3.2 Detector de formato de dados

In [4]:
def tem_formato(dados):
  format = dados.split('.')
  return format[-1]


## 3.3 Tela de menu inicial, de exibição, titulo e de caregamento de dados

In [41]:
def menu_principal():
    """Exibe o menu principal e retorna a escolha do usuário."""
    titulo("Sistema de Análise de Dados e Conversação")
    print("  Selecione uma opção:")
    print("  1. Analisar os dados")
    print("  2. Conversar com o chat")
    print("  3. consultar o historico\n")
    opcao = input("  Escolha uma opção (1 ou 2, 999 para sair.): ")
    return opcao


# função mostrar linhas de cabeçalho e centralizar o cabeçalho
def titulo(msg):
    print("-" * 50)
    print(f"{msg:^50}")
    print("-" * 50, "\n")


def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


# Função para carregar dados e extrair formato
def carregar_dados():
    caminho = input("Digite o caminho do arquivo: ")
    formato_dados = tem_formato(caminho)  # verificar o formato de arquivo

    # 1. Carregar e pré-processar os dados
    if isinstance(formato_dados, str) and formato_dados in ["csv", "json", "txt"]:
        try:
            if formato_dados == "csv":
                dados = pd.read_csv(caminho)
            elif formato_dados == "json":
                dados = pd.read_json(caminho)
            elif formato_dados == "txt":
                dados = pd.read_csv(caminho, delimiter='\t')

        except FileNotFoundError:
            return False, None
    else:
        # caso formato não seja ["csv", "json", "txt"]
        return False, None
    return True, dados


## 3.4 Conversar com Chat

In [6]:
# Função para conversar com o chat
def conversar_chat():

    # limpar tela ou console
    clear_output()

    # mostrar titulo
    titulo("Chat de conversação")

    while True:

        # receber chat de usuario
        print("\033[1;33mUser -> \033[0m")
        prompt = input("Faça sua pergunta: ")
        print()

        # verificar a saida
        if prompt == "999":
            break

        # fazer requisição e imprimir
        resposta = chat.send_message(prompt)
        print("\033[1;32mResposta do chat:\033[0m")
        display(to_markdown(f'\n\n{resposta.text}'))

        print("\n-----------------------------------------------------------\n")

## 3.5 Consultar o historico da conversa

In [7]:
def ver_historico():

    # mostrar titulo
    titulo("Historico de conversa")

    # Verificar se há conteúdo no histórico
    if chat.history:
        # Imprimir o histórico
        for message in chat.history:
            display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
            print('-------------------------------------------')
    else:
        print("O histórico está vazio.")

## 3.5 Programa principal

In [ ]:
# Loop principal

# fazendo requisição ao gemini no modo gerar historico
chat = model.start_chat(history=[])
while True:

    # limpar tela ou console
    clear_output()

    opcao = menu_principal()

    if opcao == '1':

        # limpar tela ou console
        clear_output()

        # mostrar titulo
        titulo("Sistema de Análise de Dados")
        foi_carregado, dados = carregar_dados()

        if foi_carregado is True:

            while True:

                print("\n\033[1;33mUser -> \033[0m")
                prompt = input("Digite uma pergunta sobre os dados: ")

                # verificar a saida
                if prompt == "999":
                    break

                # chamada de função responsavel por analisar dados
                insights = analisar_dados(dados, prompt)

                # imprimir o resultado
                print()
                print("\033[1;32mResposta do chat:\033[0m")
                display(to_markdown(f'\n\n{insights}'))

                print("\n-----------------------------------------------------------\n")

        else:
            #print("\n\033[1;31mERRO!\033[m")
            print("\n\033[1;31mERRO!\033[0m")
            print('Verifique se a extenão termina em ("csv", "json", "txt") ou se cominho esta correto...')
            sleep(5)

    elif opcao == '2':
        # limpar tela ou console
        clear_output()
        resp = conversar_chat()

    elif opcao == "3":
        # limpar tela ou console
        clear_output()
        ver_historico()

        # este comando so impede limpar tela ante que o usuario termina de ler historico
        input("Digite qualquer letra para sair: ")

    elif opcao == "999":
        print("\nSaindo...")
        sleep(1)
        print("Obrigado por usar o nosso sistema. Volte sempre e até logo 👋😊.")
        break

    else:
        print("  \033[1;31mOpção inválida. Por favor, escolha 1, 2 ou 3.\033[0m")
        sleep(2)
